**This is the analysis and methods for testing the null hypothesis for my project.**

Explanation of statistics to be used:

The initial analysis uses a t-test statistic to determine if the data of interest is significant in that it deviates from
the normal distribution at a level of significance.

This is determined by finding the t-value>. $$t = \frac{(\bar{x}_1 - \bar{x}_2) - D_0}{s_p \sqrt{\frac{1}{n_1}+\frac{1}{n_2}}}$$

To find the t-value first we determine the pooled standard deviation. $$ s_p = \sqrt{\frac{(n_1 - 1)s_1^2 + (n_2 - 1)s_2^2}{n_1 + n_2 - 2}}$$

We will then reject the $$H_0: \mu_1 - \mu_2 = D_o $$ if the t-value is different from the critical t-value.

The critical t-value is determined by the alpha, and is called the p value.

**Hypothesis Statements 1 & 2:**

Null Hypothesis #1: There will be no significant difference between the variance of the frequency of colexification of a sample of words
drawn from the highest quartile of colexified words and the mean levels of all of those words using ratings from the NRD and words from
the CLICS data resource.

Alternate Hypothesis #1: There will be a significant difference in the level of colexification of words with high arousal from the
mean levels of colexification of words from the full sample.

$$H_0: \mu_1 - \mu_2 = D_o$$
$$H_A: \mu_1 - \mu_2 \neq D_o$$

Null Hypothesis #2: There will be no significant difference between the variance of the frequency of colexification of a sample of words
drawn from the highest quartile of colexified words and the mean of all of those words using ratings from the NRD and words from
the CLICS data resource.

Alternative Hypothesis #2: There will be a significant difference in the level of colexification of words with high arousal from the
mean levels of colexification of words from the full sample.



$$H_0: \mu_1 - \mu_2 = D_o$$
$$H_A: \mu_1 - \mu_2 \neq D_o$$

**Exploration and assembly of Data**

Using the CLICS data as a resource I extracted the full dataframe of colexified word pairs, and recorded the frequencies of colexification.
This method as initially presented to te class also excluded sets and languages where there were no colexified pairs.

Import libraries for analysis and presentation


In [1]:
import numpy as np
import pandas as pd
# the library below is useful for estimating the progress of a for loop
# the use of tqdm will be displayed in Demo 2
from tqdm import tqdm
from itertools import product
import matplotlib.pyplot as plt
from scipy import stats as st

pd.options.mode.chained_assignment = None

Read the CLICS data from the provided .csv file into a dataframe so the information can be sorted and extracted.

In [4]:
df = pd.read_csv(r"/Users/petejohansson/PycharmProjects/cog260project/data/Colex_Data/df_all_raw.csv", low_memory=False)
df.columns = list(map(str.lower, df.columns))
df = df.drop(columns=['dataset_id', 'form_id', 'form', 'gloss_in_source', 'iso639p3code', 'mrc_word', 'kucera_francis_frequency'])
df.dtypes


clics_form               object
concepticon_id            int64
concepticon_gloss        object
ontological_category     object
semantic_field           object
variety                  object
glottocode               object
macroarea                object
family                   object
latitude                float64
longitude               float64
age_of_acquisition      float64
concreteness            float64
familiarity             float64
imagability             float64
dtype: object